In [1]:
import requests
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core import PromptTemplate

/Users/gaheeyoon/llamaindex/ch06/ch06_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
urls = [
    "https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/main/ch06/ict_japan_2024.pdf",
    "https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/main/ch06/ict_usa_2024.pdf"
]

# 각 파일 다운로드
for url in urls:
    filename = url.split("/")[-1] # URL에서 파일명 추출
    response = requests.get(url)

    with open(filename, "wb") as f:
        f.write(response.content)
    print(f"{filename} 다운로드 완료")

ict_japan_2024.pdf 다운로드 완료
ict_usa_2024.pdf 다운로드 완료


In [3]:
us_docs = SimpleDirectoryReader(
    input_files=["ict_usa_2024.pdf"]
).load_data()
jp_docs = SimpleDirectoryReader(
    input_files=["ict_japan_2024.pdf"]
).load_data()

In [4]:
print('미국 시장동향 문서의 개수:', len(us_docs))
print('일본 시장동향 문서의 개수:', len(jp_docs))

미국 시장동향 문서의 개수: 29
일본 시장동향 문서의 개수: 30


In [5]:
us_docs[5]

Document(id_='1f434c4d-91b8-4703-91d2-4c93d751cd26', embedding=None, metadata={'page_label': '6', 'file_name': 'ict_usa_2024.pdf', 'file_path': 'ict_usa_2024.pdf', 'file_type': 'application/pdf', 'file_size': 1240453, 'creation_date': '2025-07-01', 'last_modified_date': '2025-07-01'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='               6 \nⅠ. ICT 국가 산업 현황 2.ICT 정부기구  ① 국가통신정보관리청(NTIA)\n 국가통신정보관리청(NTIA)• 상무부 산하의 국가통신정보관리청(National Telecommunications and Information Administration)은 주로 통신 및 정보 정책 문제에 대하여 대통령에게 자문을 제공할 책임이 있는 행정 기관임• 국가통신정보관리청은 최근 ‘AI 업무’, ‘Open Weight AI 모델’ 등 AI 관련 정책 권장 사항 개발을 위한 대중 참여, 의견 요청 등을 시작함. 또 무선 공급망 혁신

In [6]:
print('5번 샘플에서 텍스트만 출력: ', us_docs[5].text)

5번 샘플에서 텍스트만 출력:                 6 
Ⅰ. ICT 국가 산업 현황 2.ICT 정부기구  ① 국가통신정보관리청(NTIA)
 국가통신정보관리청(NTIA)• 상무부 산하의 국가통신정보관리청(National Telecommunications and Information Administration)은 주로 통신 및 정보 정책 문제에 대하여 대통령에게 자문을 제공할 책임이 있는 행정 기관임• 국가통신정보관리청은 최근 ‘AI 업무’, ‘Open Weight AI 모델’ 등 AI 관련 정책 권장 사항 개발을 위한 대중 참여, 의견 요청 등을 시작함. 또 무선 공급망 혁신 기금을 통해 차세대 무선 기술 지원에 약 8,000만 달러(약 1,068억 4,000만 원)를 지원함[표 2] 미국 국가통신정보관리청(NTIA)구분 내용
주요 인사앨런 데이비슨(Alan Davidson)
 사라 모리스(Sarah Morris)
국가통신정보관리청 청장/차관보 국가통신정보관리청부청장/부차관보(대행)
주요 소식
‘AI 업무’ 행정명령에 대한 대중 참여 시작‣ AI 모델의 개방성 관련 검토에서 대중 참여 시작‣ 안전하고 접근성 높은 AI 정책 모색‘무선 공급망 혁신 기금’ 지원‣ 무선 공급망 혁신 기금 3차 보조금으로 약 8,000만 달러 지원‣ 차세대 무선 기술을 위한 토대 마련 목적Open Weight AI 모델에 대한 의견 요청‣ Open Weight AI 모델 관련 위험, 이점, 향후 정책에 대한 의견 요청 시작‣ 정책 권장사항 개발 목적
주요산하 조직
출처 : 미국 국가통신정보관리청


In [7]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

In [8]:
# 미국과 일본 문서를 각각의 벡터 인덱스로 생성
# 문서를 임베딩 모델을 통해 벡터화하여 인덱스를 구축
us_index = VectorStoreIndex.from_documents(us_docs, embed_model=embed_model)
jp_index = VectorStoreIndex.from_documents(jp_docs, embed_model=embed_model)

# 생성된 벡터 인덱스를 로컬 스토리지에 저장
# 이후 재사용시 다시 생성할 필요 없이 저장된 인덱스를 불러올 수 있음
us_index.storage_context.persist(persist_dir="./storage/us")
jp_index.storage_context.persist(persist_dir="./storage/jp")

# 벡터 인덱스를 쿼리 엔진으로 변환
# similarity_top_k=5는 질문과 가장 유사한 상위 5개의 문서 청크를 검색하도록 설정
us_engine = us_index.as_query_engine(similarity_top_k=5)
jp_engine = jp_index.as_query_engine(similarity_top_k=5)

In [9]:
response = us_engine.query("이 문서의 요약본을 한글로 작성해줘")
print(response)

이 문서는 2024년에 발행된 정보통신산업진흥원의 자료로, 무단 전재 및 배포를 금지하며 출처 표기 및 원본 변경 불가의 이용 규칙을 준수해야 합니다. 미국에 관한 내용이 포함되어 있으며, 참고 자료와 사이트 목록이 수록되어 있습니다.


In [10]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=us_engine,
        metadata=ToolMetadata(
            name="usa_ict",
            description=(
                "미국의 ICT 시장동향 정보를 제공합니다. 미국 ICT와 관련된 질문은 해당 도구를 사용하세요. "
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=jp_engine,
        metadata=ToolMetadata(
            name="japan_ict",
            description=(
                "일본의 ICT 시장동향 정보를 제공합니다. 일본 ICT와 관련된 질문은 해당 도구를 사용하세요."
            ),
        ),
    ),
]

In [11]:
llm = OpenAI(model="gpt-4.1", temperature=0)

agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
)

/Users/gaheeyoon/llamaindex/ch06/ch06_env/lib/python3.11/site-packages/llama_index/core/agent/react/base.py:154: DeprecationWarning: Call to deprecated class ReActAgent. (ReActAgent has been rewritten and replaced by llama_index.core.agent.workflow.ReActAgent.

This implementation will be removed in a v0.13.0 and the new implementation will be promoted to the `from llama_index.core.agent import ReActAgent` path.

See the docs for more information: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  return cls(
/Users/gaheeyoon/llamaindex/ch06/ch06_env/lib/python3.11/site-packages/deprecated/classic.py:184: DeprecationWarning: Call to deprecated class AgentRunner. (AgentRunner has been deprecated and is not maintained.

This implementation will be removed in a v0.13.0.

See the docs for more information on updated agent usage: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  return old_new1(cls, *args, **kwargs)


In [12]:
print('기본 프롬프트')
print(agent.get_prompts()['agent_worker:system_prompt'].template)

기본 프롬프트
You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}


## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

NEVER surround your response with markdown code markers. You may use code markers within your respo

In [13]:
react_system_header_str = """
당신은 질문에 답변하는 것부터 요약 제공, 기타 여러 유형의 분석까지 다양한 작업을 돕기 위해 설계되었습니다.

## 도구
당신은 다양한 도구에 접근할 수 있습니다. 현재 작업을 완료하기 위해 적절하다고 판단되는 순서로 도구를 사용하는 것은 당신의 책임입니다. 이를 위해 작업을 하위 작업으로 나누고, 각 하위 작업에 다른 도구를 사용할 필요가 있을 수 있습니다.

당신은 다음 도구들에 접근할 수 있습니다:
{tool_desc}

## 출력 형식
질문에 답변하기 위해 다음 형식을 사용하십시오.

###
Thought: I need to use a tool to help me answer the question.
Action: 도구 이름 (사용할 도구 중 하나인 {tool_names})
Action Input: 도구에 대한 입력을 JSON 형식으로 제공하십시오. 예: {{\"input\": \"hello world\", \"num_beams\": 5}}
###

항상 'Thought'로 시작하십시오.

'Action Input'에서는 올바른 JSON 형식을 사용하십시오. 이렇게 쓰지 마십시오: {{'input': 'hello world', 'num_beams': 5}}.

이 형식이 사용되면, 사용자는 다음 형식으로 응답할 것입니다:

###
Observation: 도구 응답
###

이 형식을 계속 반복하여 더 이상 도구를 사용하지 않고 질문에 답변할 수 있을 만큼 충분한
정보를 얻을 때까지 진행하십시오. 그 시점에서는 반드시 다음 두 가지 형식 중 하나로 응답해야
합니다:

###
Thought: I can answer without using any more tools.
Answer: [여기에 답변을 작성하세요]
###

###
Thought: I cannot answer the question with the provided tools.
Answer: 죄송합니다. 해당 질문에 답변할 수 없습니다.
###

## 추가 규칙
- 답변은 반드시 질문에 도달하기까지의 과정을 설명하는 순차적인 항목들로 구성되어야 합니다. 여기에는 이전 대화의 내용이 포함될 수 있습니다.
- 각 도구의 함수 서명을 반드시 준수해야 하며, 함수가 인수를 기대할 경우 인수를 생략하지 마십시오.
- 답변은 반드시 '한글'로 상세하게 작성되어야 합니다.
- 질문에 대한 답변만 작성하세요. 질문과 관계없는 검색을 수행하지 마십시오. 이는 매우 중요합니다.
- Please follow the thought-action-input format.
- 하나의 질문에 많은 검색어가 포함되어져 있는 것처럼 보인다면 검색어를 나누어서 순차적으로 검색하십시오. 더 좋은 답변을 얻을 수 있을 것입니다.
- 도구를 사용하여 답변할 수 있는 주제라면 반드시 도구를 사용하시기 바랍니다. 이는 매우 중요합니다.
- 당신이 도구 없이 답변하는 것은 도구의 주제와 완전히 다른 주제의 질문이 들어왔을 때 뿐입니다. 도구와 연관된 질문이라면 반드시 도구를 호출하십시오. 이는 매우 중요하며 당신이 지켜야 할 1순위의 우선사항입니다.

## 현재 대화
아래는 인간과 어시스턴트 메시지가 교차되어 있는 현재 대화 내용입니다.

"""

In [14]:
react_system_prompt = PromptTemplate(react_system_header_str)
agent.update_prompts({"agent_worker:system_prompt": react_system_prompt})
agent.reset()

In [15]:
response = agent.chat("한국과 미국의 ICT 기관 협력 사례")

> Running step 2f64ddd5-8920-404e-b2e3-ca33744afc15. Step input: 한국과 미국의 ICT 기관 협력 사례
Thought: 미국의 ICT 기관 협력 사례에 대해 알아보기 위해 관련 정보를 검색해야 합니다.
Action: usa_ict
Action Input: {'input': '한국과 미국의 ICT 기관 협력 사례'}
Observation: Korea and the United States have been noted for their ICT institutional cooperation.
> Running step 2d957990-5a98-4486-a302-ec8217f6bfb4. Step input: None
Thought: 미국 ICT 기관과의 협력 사례에 대한 구체적인 정보가 필요하므로, 추가적으로 일본 ICT 기관과의 협력 사례와 비교할 필요는 없다고 판단됩니다. 따라서, 도구를 더 이상 사용할 필요 없이 지금까지의 정보를 바탕으로 답변을 정리할 수 있습니다.
Answer: 1. 한국과 미국은 ICT(정보통신기술) 분야에서 다양한 기관 협력 사례를 보유하고 있습니다.  
2. 대표적으로, 한국의 과학기술정보통신부(MSIT)와 미국의 상무부(Department of Commerce), 연방통신위원회(FCC) 등 주요 ICT 정책기관 간에 정기적인 정책 대화와 협력 MOU(양해각서) 체결이 이루어지고 있습니다.  
3. 양국은 5G, 6G 등 차세대 이동통신 기술 개발, 인공지능(AI), 사이버보안, 디지털 전환 등 첨단 ICT 분야에서 공동 연구 및 실증사업을 추진하고 있습니다.  
4. 예를 들어, 2021년 한미 정상회담을 계기로 ‘한미 ICT 정책 포럼’이 개최되어, 양국 정부 및 민간 전문가들이 ICT 정책, 표준화, 기술협력 방안 등을 논의하였습니다.  
5. 또한, 한국전자통신연구원(ETRI)과 미국의 국립표준기술연구소(NIST) 등 연구기관 간에도 공동연구, 인력 교류, 기술 표준화 협력이 활발

In [16]:
response = agent.chat("미국과 일본의 ICT 주요 정책의 공통점과 차이점을 설명해줘.")

> Running step cc6adc69-7c60-4ff8-8711-35c26cb18e23. Step input: 미국과 일본의 ICT 주요 정책의 공통점과 차이점을 설명해줘.
Thought: 미국과 일본의 ICT 주요 정책의 공통점과 차이점을 파악하기 위해 각각의 국가별 ICT 정책 동향을 확인해야 합니다. 우선 미국의 ICT 주요 정책을 확인한 후, 일본의 ICT 주요 정책을 확인하고, 마지막으로 공통점과 차이점을 정리하겠습니다.
Action: usa_ict
Action Input: {'input': '미국의 ICT 주요 정책 동향'}
Observation: 미국의 ICT 주요 정책 동향은 국가 스펙트럼 전략을 중심으로 진행되고 있습니다. 이에 따라 미국 정부는 신규 주파수 공급 전략을 발표하고 스펙트럼 정책 현대화를 위한 대통령 각서를 발표했습니다. 또한, 스펙트럼 파이프라인 식별, 스펙트럼 관련 갈등 해결, 스펙트럼 쉐어링 기능 개발 등을 통해 첨단 기술에 중점을 두고 있습니다. 이를 통해 국가 스펙트럼 전략을 통해 조정력을 개선하고 혁신을 촉진하며 효율적인 스펙트럼 사용을 위한 목표를 설정하고 있습니다.
> Running step 080e9b1d-77ef-40b8-acdc-0c8f248bbff3. Step input: None
Thought: 이제 일본의 ICT 주요 정책 동향을 확인하여, 미국과의 공통점과 차이점을 비교할 수 있는 정보를 수집하겠습니다.
Action: japan_ict
Action Input: {'input': '일본의 ICT 주요 정책 동향'}
Observation: Japan's ICT major policies include initiatives such as promoting domestic investment in strategic areas like semiconductor, electric vehicles, green steel, green chemicals, and SAF. Additionally, there are 